In [1]:
import random
import numpy as np
import re
import pandas as pd

In [5]:
# read in data and doing kmeans using max's code
bert_embs = np.load("./data/st_short_desc_summaries.npz")
bert_embs = [bert_embs[f] for f in bert_embs.files]

In [7]:
tender_refs = open("./data/tender_references.txt", "r")
content = ""
for line in tender_refs:
    content = line
    break

In [8]:
content = content.replace("[", "")
content = content.replace("'", "")
content = content.replace(",", "")
refs = content.split(" ")
print(len(refs))
print(len(bert_embs))

20619
20619


In [9]:
from sklearn.cluster import KMeans
from sklearn import preprocessing
kmeans = KMeans(n_clusters = 2000, n_init = 1).fit_predict(preprocessing.normalize(bert_embs))

In [11]:
import pandas as pd
tenders_structured_path = r"./data/UpdatedAgainTenders.xlsx"

tenders_structured = pd.read_excel(tenders_structured_path)
tenders_structured = tenders_structured[["Reference Number", "Contract Title", "Description"]].dropna(subset=["Reference Number"])

In [12]:
clusters = pd.DataFrame({"Reference Number": refs, "cluster": kmeans})
clusters = pd.merge(clusters, tenders_structured, on = "Reference Number")

In [13]:
from bs4 import BeautifulSoup
def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    cleaned = soup.get_text().replace('\xa0', ' ')
    cleaned = ' '.join(cleaned.split())
    return cleaned

docs = []
for index, row in clusters.iterrows():
    title = row["Contract Title"]
    desc = remove_html_tags(row["Description"])
    clusters.at[index, "Description"] = desc
    docs.append(title + ". " + desc)

In [15]:
clusters.loc[clusters["cluster"] == 200 , ["Reference Number", "Contract Title", "Description"]] 

,Reference Number,Contract Title,Description
9421,DMIRS22099,DMIRS22099 - Provision of .Net Developer Servi...,DMIRS22099 - Provision of .Net Developer Servi...
9457,DMIRS22026,Provision of .Net Developer Services - ASM,Provision of .Net Developer Services - ASM
9597,DMIRS21606,Provision of .Net Developer Services,Provision of .Net Developer Services
9668,DMIRS20382,Provision of ASM .Net Developer Services,Provision of ASM .Net Developer Services
9707,DMIRS20334,Provision of ASM .NET Developer Services,Provision of ASM .NET Developer Services
9807,DMIRS160220,Provision of ASM .Net Developer Services,DMIRS160220 - Provision of ASM .Net Developer ...
9841,DMIRS1481119,Provision of .NET and Angular Development Serv...,DMIRS1481119 Provision of .NET and Angular Dev...


In [16]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
tender_tfidf = TfidfVectorizer(stop_words="english",
                               min_df=1, max_df = 0.3)
tender_tfidf.fit(docs)

TfidfVectorizer(max_df=0.3, stop_words='english')

In [17]:
from sklearn.decomposition import NMF

def display_topic_nmf(n_cluster):
    subset_docs = [docs[i] for i in list(np.where(clusters["cluster"] == n_cluster)[0])]
    tfidf = tender_tfidf.transform(subset_docs)
    
    nmf = NMF(n_components=1, max_iter=20)
    nmf_fitted = nmf.fit(tfidf)

    topics = nmf_fitted.components_
    topic_keywords = [tender_tfidf.get_feature_names_out()[topic.argsort()[-5:]] for topic in topics]

    print(topic_keywords)
    
    return clusters.loc[clusters["cluster"] == n_cluster, ["Reference Number", "Contract Title", "Description"]]


In [20]:
display_topic_nmf(9)

[array(['hudson', 'response', 'security', 'alarm', 'patrols'], dtype=object)]


C:\Users\yfr\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(
C:\Users\yfr\anaconda3\lib\site-packages\sklearn\decomposition\_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 20 reached. Increase it to improve convergence.
  warnings.warn(


,Reference Number,Contract Title,Description
13447,EMHS20205761,Carpark Emergency Intercoms at Royal Perth Hos...,Carpark Emergency Intercoms at Royal Perth Hos...
18396,MAPD2021012,Provision of Services for a Combined Mobile Se...,Provision of Services for a Combined Mobile Se...
18438,MAPD2021005,Provision of after-hours security patrols for ...,Provision of after-hours security patrols for ...
24466,WACHS202311329,Security patrols and alarm response for Hudson...,The Customer is seeking offers for the provisi...
24467,WACHS202311329,Security patrols and alarm response for Hudson...,The Customer is seeking offers for the provisi...
24468,WACHS202311329,Security patrols and alarm response for Hudson...,The Customer is seeking offers for the provisi...
24928,WACHS20217671,STATIC GUARD SERVICES NORTHAM HOSPITAL,Guard Services at Northam Hospital
24929,WACHS20217671,STATIC GUARD SERVICES NORTHAM HOSPITAL,Guard Services at Northam Hospital
